In [1]:
import os
import pandas as pd
import time

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest


In [2]:
main_path = "data"

path_to_idle = pd.DataFrame()
path_to_running = pd.DataFrame()
path_to_stairs = pd.DataFrame()
path_to_walking = pd.DataFrame()

for i in os.listdir(main_path):
    i_path = os.path.join(main_path, i)
    if os.path.isdir(i_path):
        for file in os.listdir(i_path):
            end_path = os.path.join(i_path, file)        
            
            df = pd.read_csv(end_path)
            df['activity_folder'] = i  
            
            if i == 'idle':
                path_to_idle = pd.concat([path_to_idle, df], ignore_index=True)
            elif i == 'running':
                path_to_running = pd.concat([path_to_running, df], ignore_index=True)
            elif i == 'stairs':
                path_to_stairs = pd.concat([path_to_stairs, df], ignore_index=True)
            elif i == 'walking':
                path_to_walking = pd.concat([path_to_walking, df], ignore_index=True)

data = pd.concat([path_to_idle, 
                  path_to_running, 
                  path_to_stairs, 
                  path_to_walking], ignore_index=True)
    

Запишемо остаточний результат в окремий файл csv, щоб з ним було зручно працювати

In [3]:
path_to_csv = 'indicators.csv'
data.to_csv(path_to_csv, index=False)

result_data = pd.read_csv('indicators.csv')
result_data

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity_folder
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle
...,...,...,...,...
193855,5.109226,-15.452178,-1.470040,walking
193856,6.373365,-11.741165,-8.226476,walking
193857,3.289633,-9.993398,-0.383072,walking
193858,-2.978387,-3.050213,1.273715,walking


In [4]:
X = result_data.drop('activity_folder', axis=1)
X.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,1.000776,4.616021,8.576031
1,0.718261,4.209007,8.446744
2,-0.909797,-0.282516,9.203311
3,5.099650,0.148441,8.418014
4,1.762132,-0.162806,9.251195


In [5]:
y = result_data['activity_folder']
y.head()

0    idle
1    idle
2    idle
3    idle
4    idle
Name: activity_folder, dtype: object

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Тепер визначимо середні значення наших данних

In [7]:
X_train['average over time'] = X_train.mean(axis=1)
X_train.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,average over time
57654,-2.212242,7.522582,-1.924938,1.128467
12457,0.296881,0.028730,9.792285,3.372632
177098,2.288857,-10.242394,-1.647211,-3.200249
191995,-3.490746,-9.471462,-1.197101,-4.719770
118812,-0.191536,5.980716,-0.967257,1.607308


In [8]:
X_test['average over time'] = X_test.mean(axis=1)
X_test.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,average over time
171248,-1.991976,-9.342175,-2.743755,-4.692635
58417,-0.957681,11.698070,5.410896,5.383762
54186,1.958457,-4.271256,0.459687,-0.617704
131711,-8.485051,16.735470,-5.918466,0.777318
27960,-0.531513,6.009446,7.196971,4.224968


Створимо 2 моделі машинного навчання:

In [9]:
# модель випадкогово лісу
random_forest_model = RandomForestClassifier(class_weight='balanced')

start_time = time.time()
random_forest_model.fit(X_train, y_train)
end_time = time.time()

training_time_seconds = end_time - start_time
training_time_minutes, training_time_seconds = divmod(training_time_seconds, 60)
print("Model training time:", int(training_time_minutes), "minutes", int(training_time_seconds), "seconds")

Model training time: 0 minutes 21 seconds


In [10]:
# модель машинного навчання методом опорних векторів
support_vector_method = SVC()

start_time = time.time()
support_vector_method.fit(X_train, y_train)
end_time = time.time()

training_time_seconds = end_time - start_time
training_time_minutes, training_time_seconds = divmod(training_time_seconds, 60)
print("Model training time:", int(training_time_minutes), "minutes", int(training_time_seconds), "seconds")

Model training time: 7 minutes 44 seconds


Тепер зробимо прогноз на тестових данних 

In [11]:
start_time = time.time()
random_forest_model_prediction = random_forest_model.predict(X_test)
end_time = time.time()

prediction_time_seconds = end_time - start_time
prediction_time_minutes, prediction_time_seconds = divmod(prediction_time_seconds, 60)
print("Model training time:", int(prediction_time_minutes), "minutes", int(prediction_time_seconds), "seconds")

Model training time: 0 minutes 0 seconds


In [12]:
start_time = time.time()
support_vector_method_prediction = support_vector_method.predict(X_test)
end_time = time.time()

prediction_time_seconds = end_time - start_time
prediction_time_minutes, prediction_time_seconds = divmod(prediction_time_seconds, 60)
print("Model training time:", int(prediction_time_minutes), "minutes", int(prediction_time_seconds), "seconds")

Model training time: 5 minutes 22 seconds


Тепер розрахуємо точність і зробимо звіт класифікації. Ми будемо порівнювати прогнози моделі з фактичними мітками класів тестових данних.

In [13]:
random_forest_model_prediction_precision = accuracy_score(y_test, random_forest_model_prediction)
print("accuracy for the vicinity forest model")
print(random_forest_model_prediction_precision)

random_forest_model_prediction_summary = classification_report(y_test, random_forest_model_prediction)
print("classification report for the vicinity forest model")
print(random_forest_model_prediction_summary)

accuracy for the vicinity forest model
0.9995460641700196
classification report for the vicinity forest model
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      7786
     running       1.00      1.00      1.00     25542
      stairs       1.00      0.99      1.00      1290
     walking       1.00      1.00      1.00     13847

    accuracy                           1.00     48465
   macro avg       1.00      1.00      1.00     48465
weighted avg       1.00      1.00      1.00     48465



In [14]:
support_vector_method_prediction_precision = accuracy_score(y_test, support_vector_method_prediction)
print("accuracy for the SVM model")
print(support_vector_method_prediction_precision)

support_vector_method_prediction_summary = classification_report(y_test, support_vector_method_prediction)
print("classification report for the SVM model")
print(support_vector_method_prediction_summary)

accuracy for the SVM model
0.8913442690601465
classification report for the SVM model
              precision    recall  f1-score   support

        idle       0.95      0.99      0.97      7786
     running       0.93      0.90      0.92     25542
      stairs       1.00      0.00      0.01      1290
     walking       0.80      0.90      0.85     13847

    accuracy                           0.89     48465
   macro avg       0.92      0.70      0.68     48465
weighted avg       0.90      0.89      0.88     48465



## Висновки:

1. Модель випадкового лісу має більшу точність, вона наближена до 100%, точність моделі опорних векторів менш точна, вона складає 89%. Тобто помилок буде більше в моделі опорних векторів.
2. Якщо ми проаналізуємо звіти по 2-м методам, то:
   - у моделі випадкового лісу значення precision та f1-score мають високий показник = 1. Це означає, що модель дуже гарно передбачає усі класи, не допускає помилок і практично не пропускає об'єкти ні одного класу (значення recall).
   - у моделі опорних векторів гірші результати по звіту, параметр precision показує що точність не ідеальна, і передбачення будуть з помилками. Требі звернути увагу на клас stairs - recall = 0, отже ні один з прикладів stairs не був коректно класифікований моделлю.
3. Пунктом хочу порівняти час для використання двох моделей - модель випадкового лісу значно швидша за модель опорних векторів. 

Модель випадкового лісу дає більш точні результати. 